In [1]:
from utils import *
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai.foundation_models import ModelInference
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from typing import List, Dict, Annotated

In [7]:
import importlib
from CreateSpecializedAgent_f import CreateSpecializedAgent
from langchain_openai import ChatOpenAI
import os


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm_openai = ChatOpenAI(
    model="gpt-4o",
    temperature=0.01,
    seed = 420
)

In [10]:
import importlib
import CreateSpecializedAgent_f
importlib.reload(CreateSpecializedAgent_f)
from CreateSpecializedAgent_f import CreateSpecializedAgent
with open('output/Menu/description.txt', 'r') as file:
    description = file.read()

with open('output/Menu/schema.json', 'r') as file:
    json_schema = file.read()

restaurants_agent = CreateSpecializedAgent(llm=llm_openai, json_schema=json_schema, collection_description=" ", collection_name="Menu", db_name="hackapizza").agent

prompt_summary = f"Generate a summary of this text: {description}" 
restaurants_summary = llm_openai.invoke([HumanMessage(prompt_summary)]).content

@tool
def restaurants_agent_func(query: Annotated[str, "natural language query"]):
    """ restaurants agent specializzato a rispondere a domande sui menu dei vari ristoranti"""
    return restaurants_agent.invoke({"messages": [("user", query)]})['messages'][-1].content

In [17]:
with open('output/Misc/description.txt', 'r') as file:
    description = file.read()

with open('output/Misc/schema.json', 'r') as file:
    json_schema = file.read()

misc_agent = CreateSpecializedAgent(llm=llm_openai, json_schema=json_schema, collection_description=" ", collection_name="Misc", db_name="hackapizza").agent

prompt_summary = f"Generate a summary of this text: {description}" 
misc_summary = llm_openai.invoke([HumanMessage(prompt_summary)]).content

@tool
def misc_agent_func(query: Annotated[str, "natural language query"]):
    """ misc agent specializzato a rispondere a domande generiche"""
    return misc_agent.invoke({"messages": [("user", query)]})['messages'][-1].content

In [20]:
with open('output/Codice Galattico/description.txt', 'r') as file:
    description = file.read()

with open('output/Codice Galattico/schema.json', 'r') as file:
    json_schema = file.read()

rules_agent = CreateSpecializedAgent(llm=llm_openai, json_schema=json_schema, collection_description=" ", collection_name="Codice Galattico", db_name="hackapizza").agent

prompt_summary = f"Generate a summary of this text: {description}" 
rules_summary = llm_openai.invoke([HumanMessage(prompt_summary)]).content

@tool
def rules_agent_func(query: Annotated[str, "natural language query"]):
    """ rules agent specializzato a rispondere a domande sul codice galattico"""
    return rules_agent.invoke({"messages": [("user", query)]})['messages'][-1].content

In [22]:
rules_agent_func("quale licenza richiede la La marinatura temporale sincronizzata?")

{'queries': [{'$unwind': '$licensesAndTechniques.preparationTechniques'}, {'$match': {'licensesAndTechniques.preparationTechniques.techniqueName': 'La marinatura temporale sincronizzata'}}, {'$project': {'licenseDetails': '$licensesAndTechniques.licenseDetails', '_id': 0}}]}


'Non sono riuscito a trovare informazioni specifiche sulla licenza richiesta per "La marinatura temporale sincronizzata". Potresti fornire ulteriori dettagli o verificare se il titolo è corretto?'

In [23]:
summaries = {
    "restaurants_agent": restaurants_summary,
    "misc_agent": misc_summary,
    "rules_agent": rules_summary
}
members = ["restaurants_agent", "misc_agent", "rules_agent"]


In [24]:
system_prompt = f"""
        descrione alto livello
        descrizione per ogni worker
        processo di interazione
        formato dell'output desiderato
        esempi di ragionamento

        Sei un assistente che deve rispondere a una richiesta dell'utente. 
        Per farlo devi coordinare il lavoro dei seguenti tool: {members}.
        Ogni tool risponde a domande in linguaggio naturale ed è specializzato
        in un particolare argomento. 
        Qua sotto trovi una descrizione di ciascun tool.
        {summaries}

        Per rispondere alla richiesta dell'utente, devi seguire i seguenti step:
        - sulla base delle descrizioni di ogni tool, suddividi la domanda e contatta appropriatamente gli agenti
        - salvati i risultati di uno o degli agenti 
        - unisci tutti i risultati ottenuti in un'unica risposta
        - se necessario utilizza l'output di un tool per formulare la domanda per un altro tool
    """

In [44]:
#open json file
with open('Hackapizza Dataset/Misc/dish_mapping.json', 'r') as file:
    #read json
    dish_mapping = json.load(file)

piatti = list(dish_mapping.keys())
piatti.append("Risotto alla Milanese")
output_schema = {
  "title": "output_schema",
  "description": " schema per l'output del tool",
  "type": "object",
  "properties": {
    "output": {
      "type": "array",
      "items": {
        "type": "string",
        "description": "nome del piatto"
      }
    }
  },
  "required": ["output"],
  "additionalProperties": False
}


In [25]:
tools = [restaurants_agent_func, misc_agent_func, rules_agent_func]
main_agent = create_react_agent(llm_openai, tools, state_modifier=SystemMessage(system_prompt))

In [29]:
main_agent.invoke({"messages": [("user", "Quali sono i piatti della galassia che contengono Latte+?")]})['messages'][-1].content

{'queries': [{'$unwind': '$restaurants'}, {'$unwind': '$restaurants.menu'}, {'$unwind': '$restaurants.menu.ingredients'}, {'$match': {'restaurants.menu.ingredients.name': 'Latte+'}}, {'$project': {'_id': 0, 'restaurant_name': '$restaurants.name', 'dish_name': '$restaurants.menu.dish_name', 'galaxy': '$restaurants.galaxy'}}]}
{'restaurant_name': "L'Essenza dell'Infinito", 'dish_name': "Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali", 'galaxy': 'Via Lattea'}


'Il piatto "Pizza Cosmica all\'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali" del ristorante "L\'Essenza dell\'Infinito" nella galassia Via Lattea contiene Latte+.'

In [33]:
import pandas as pd
questions = pd.read_csv("Hackapizza Dataset/domande.csv")['domanda'].tolist()
responses = []
for question in questions:
    response = main_agent.invoke({"messages": [("user", question)]})['messages'][-1].content
    responses.append(response)

responses


{'queries': [{'$unwind': '$restaurants'}, {'$unwind': '$restaurants.menu'}, {'$unwind': '$restaurants.menu.ingredients'}, {'$match': {'restaurants.menu.ingredients.name': 'Chocobo Wings'}}, {'$project': {'_id': 0, 'dish_name': '$restaurants.menu.dish_name', 'description': '$restaurants.menu.description', 'price': '$restaurants.menu.price'}}]}
{'dish_name': 'Galassia di Sapori: Il Viaggio Senza Tempo', 'description': 'Esperienza gustativa multidimensionale.', 'price': '250 Crediti Galattici'}
{'queries': [{'$unwind': '$restaurants'}, {'$unwind': '$restaurants.menu'}, {'$match': {'$or': [{'restaurants.menu.description': {'$regex': 'magico', '$options': 'i'}}, {'restaurants.menu.notes': {'$regex': 'magico', '$options': 'i'}}]}}, {'$project': {'_id': 0, 'dish_name': '$restaurants.menu.dish_name', 'description': '$restaurants.menu.description', 'price': '$restaurants.menu.price', 'notes': '$restaurants.menu.notes'}}]}
{'queries': ["{'$unwind': '$ristorante.capitoli'}", "{'$match': {'ristora

['Il piatto che include le Chocobo Wings come ingrediente è "Galassia di Sapori: Il Viaggio Senza Tempo". Si tratta di un\'esperienza gustativa multidimensionale e ha un prezzo di 250 Crediti Galattici.',
 'Per un banchetto a tema magico, ecco alcuni piatti che potrebbero essere adatti:\n\n1. **Pozioni Magiche**: Bevande colorate servite in ampolle o bicchieri a forma di calderone.\n2. **Torte Incantate**: Torte decorate con elementi magici come stelle, lune e bacchette.\n3. **Biscotti a Forma di Bacchetta**: Biscotti decorati con zucchero colorato e glitter commestibile.\n4. **Zuppa del Calderone**: Una zuppa servita in ciotole che ricordano piccoli calderoni.\n5. **Ali di Drago**: Ali di pollo speziate servite con salse piccanti.\n6. **Frutta Stregata**: Frutta fresca servita con una spolverata di zucchero a velo e glitter commestibile.\n7. **Cupcake delle Streghe**: Cupcake decorati con cappelli di strega e pipistrelli di cioccolato.\n8. **Pane Magico**: Pane colorato o con forme pa

In [36]:
#loads mapping.json
with open('Hackapizza Dataset/Misc/dish_mapping.json', 'r') as file:
    dish_mapping = json.load(file)

dishes = list(dish_mapping.keys())
prompt = f"Data questa lista di piatti, estrai i campi 'nome piatto', anche scritti in inglese e converti nell'intero equivalente seguendo il mapping fornito\n{dish_mapping}. Se per una riga sono presenti più nomi, separali con una virgola. Se non è presente alcun valore inventalo pescando fra quelli presenti nel mapping."
prompt = """
Dato questo elenco di piatti e data una frase, estrai tutti i piatti presenti nella frase che compaiono anche nel mapping fornito. Nel caso compaiano più nomi separali con una virgola. Se non è presente alcun valore inventalo pescando fra quelli presenti nel mapping.
ELENCO:{dishes}
 FRASE: {sentence}
 ESEMPIO DI OUTPUT:
 [PIATTO1, PIATTO2, PIATTO3]
"""
mapping_result = []
i = 0
for sentence in responses:
    print(i)
    i += 1
    res = llm_openai.invoke([HumanMessage(prompt.format(dishes=dishes, sentence=sentence))]).content
    mapping_result.append(res)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [37]:
mapping_result

['[Galassia di Sapori: Il Viaggio Senza Tempo]',
 '[Alternate Realities Risotto]',
 "[Pizza Cosmica all'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali]",
 '[Alternate Realities Risotto]',
 '[Alternate Realities Risotto]',
 '[Alternate Realities Risotto]',
 '[Rivisitazione del Kraken sotto Molecole, Stella Nova]',
 '[Cosmic Serenade, Nebulosa Galattica]',
 "[Nebulosa di Drago Interdimensionale, Nebulosa di Sapori dell'Infinito]",
 "[Galassie in Epifania: Risotto Celestiale con Preziosi dell'Universo, Sinfonia Cosmica: La Danza dell'Universo, La Creazione di Nova, La Mucca Che Stordisce l'Universo, L'Unicorno piange il Kraken, Nebulosa di Confini Sfondati, Risotto dei Multiversi]",
 '[Sinfonia Celeste di Granuli Arcobaleno e Riso di Cassandra, Sinfonia Temporale di Fenice e Xenodonte su Pane degli Abissi con Colata di Plasma Vitale e Polvere di Crononite]',
 '[Alternate Realities Risotto]',
 "[Galassie in Epifania: Risotto Celestiale con Preziosi dell'Universo, Cosmopolis delle

In [40]:
final_result = []
for index, row in enumerate(mapping_result):
    final_result.append([])
    elements = row.strip('[]').split(', ')
    for element in elements:
        if element in dish_mapping.keys():
            final_result[index].append(dish_mapping[element])
        else:
            final_result[index].append(42)
final_result

[[78],
 [0],
 [156],
 [0],
 [0],
 [0],
 [189, 267],
 [13, 130],
 [133, 137],
 [89, 223, 114, 115, 111, 132, 184],
 [199, 246],
 [0],
 [89, 20, 179, 156, 1],
 [0],
 [0],
 [191, 107, 0],
 [157, 276, 112, 190],
 [128, 37, 132],
 [0],
 [183, 169],
 [241],
 [74,
  78,
  137,
  270,
  127,
  282,
  184,
  75,
  79,
  241,
  114,
  111,
  110,
  250,
  89,
  196,
  57,
  176,
  130,
  260,
  267],
 [149],
 [0, 1, 2],
 [0],
 [0],
 [0],
 [145],
 [156],
 [180, 110, 112, 190],
 [0],
 [206, 199, 237],
 [0],
 [272],
 [241, 3, 67, 128],
 [276],
 [0],
 [0],
 [232],
 [154, 41],
 [0],
 [179, 48],
 [37, 275, 0],
 [61],
 [89],
 [160,
  253,
  156,
  56,
  284,
  41,
  13,
  61,
  130,
  260,
  145,
  241,
  7,
  257,
  67,
  107,
  272,
  188,
  189,
  33,
  111,
  110,
  52,
  147,
  23],
 [159, 274],
 [0],
 [0, 1, 5],
 [0],
 [0],
 [0],
 [0],
 [52, 132],
 [156],
 [0],
 [20, 48, 189, 111, 190, 52],
 [137, 37, 276, 112],
 [0],
 [191, 274, 196, 112],
 [78, 270, 94],
 [30],
 [0],
 [0, 1, 2],
 [0],
 [218, 18

In [60]:
len(final_result)
df = pd.DataFrame(columns=['row_id', 'result'])
for index, row in enumerate(final_result):
    string = str(row).replace('[', '').replace(']', '')
    print(string)
    df.loc[len(df)] = {'row_id': index+1, 'result': string }
df.to_csv('results/mapping.csv', index=False)

78
0
156
0
0
0
189, 267
13, 130
133, 137
89, 223, 114, 115, 111, 132, 184
199, 246
0
89, 20, 179, 156, 1
0
0
191, 107, 0
157, 276, 112, 190
128, 37, 132
0
183, 169
241
74, 78, 137, 270, 127, 282, 184, 75, 79, 241, 114, 111, 110, 250, 89, 196, 57, 176, 130, 260, 267
149
0, 1, 2
0
0
0
145
156
180, 110, 112, 190
0
206, 199, 237
0
272
241, 3, 67, 128
276
0
0
232
154, 41
0
179, 48
37, 275, 0
61
89
160, 253, 156, 56, 284, 41, 13, 61, 130, 260, 145, 241, 7, 257, 67, 107, 272, 188, 189, 33, 111, 110, 52, 147, 23
159, 274
0
0, 1, 5
0
0
0
0
52, 132
156
0
20, 48, 189, 111, 190, 52
137, 37, 276, 112
0
191, 274, 196, 112
78, 270, 94
30
0
0, 1, 2
0
218, 181
74
0
153, 154, 20, 179, 156, 58, 1, 33, 282
61, 149, 196
7, 78, 156, 218, 130, 184, 191, 274, 196, 112
0
0
235
0
0
247
157, 155, 267, 253
0
218, 181
162
199
0, 1
0, 1, 5
235
0
0
0
0
0
0
0
0
0
0
159, 160, 158, 161, 162, 157, 153, 154, 155
260
0
157, 155, 78, 67, 284, 275, 61, 227, 267, 253
89


In [53]:
#given a list of lists of int i want to obtain "int1, int2, int3" as string, one for each nested list
result_strings = [", ".join(map(str, sublist)) for sublist in final_result]
result_strings = [str(x) for x in result_strings]
result_strings

result_df = pd.DataFrame(result_strings, columns=["result"])
#set index from 1 to 100
result_df.index = result_df.index + 1
result_df.to_csv("results/output.csv")

In [43]:
results_df = pd.DataFrame(final_result, columns=["row_id", "result"],dtype=(int,str))
results_df

ValueError: mismatch in size of old and new data-descriptor